<h1>Exercise 07</h1>
<p>Pareto principle: For many outcomes, roughly 80% of consequences come from 20% of causes. In this problem , tackle "80% of your sales comes from 20% of your products/services".</p>


In [1]:
import sqlite3
import pandas as pd
import os

db_name="exercise07.db"
conn=sqlite3.connect(db_name)
cursor=conn.cursor()

with open("Day7_data.txt", 'r') as sql_file:
    sql_script = sql_file.read()

# Executing table creation and inserts at once
try:
    cursor.executescript(sql_script)
    conn.commit()
    print("Database populated successfully!")
except sqlite3.Error as e:
    print(f"An error occurred: {e}")

#Verify if table was created
df = pd.read_sql("SELECT * FROM orders LIMIT 10", conn)
print(df)


Database populated successfully!
   row_id        order_id  order_date   ship_date       ship_mode customer_id  \
0  1364.0  US-2021-155425  2021-11-10  2021-11-11     First Class    AB-10600   
1  1365.0  US-2021-155425  2021-11-10  2021-11-11     First Class    AB-10600   
2  1366.0  US-2021-155425  2021-11-10  2021-11-11     First Class    AB-10600   
3  1367.0  US-2021-155425  2021-11-10  2021-11-11     First Class    AB-10600   
4  1368.0  US-2021-155425  2021-11-10  2021-11-11     First Class    AB-10600   
5  1369.0  CA-2021-133249  2021-07-08  2021-07-11     First Class    SZ-20035   
6  1370.0  US-2019-103471  2019-12-24  2019-12-28  Standard Class    JR-15670   
7  1371.0  US-2019-103471  2019-12-24  2019-12-28  Standard Class    JR-15670   
8  1372.0  CA-2021-136672  2021-03-07  2021-03-12  Standard Class    MG-17890   
9  1373.0  US-2018-157021  2018-04-01  2018-04-06    Second Class    KM-16720   

      customer_name      segment        country              city  ...  \
0

In [2]:
queryexec = """
WITH product_wise_sales AS (
    SELECT product_id, SUM(sales) AS product_sales 
    FROM orders GROUP BY product_id
), 
cal_sales AS (
    SELECT 
        product_id, 
        product_sales, 
        SUM(product_sales) OVER (ORDER BY product_sales DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_sales, 
        0.8 * SUM(product_sales) OVER () AS total_sales 
    FROM product_wise_sales
) 
SELECT * FROM cal_sales WHERE running_sales <= total_sales
"""

df = pd.read_sql(queryexec, conn)
print(df)
conn.close()

          product_id  product_sales  running_sales   total_sales
0    TEC-CO-10004722      61599.880      61599.880  1.837761e+06
1    OFF-BI-10003527      27453.384      89053.264  1.837761e+06
2    TEC-MA-10002412      22638.500     111691.764  1.837761e+06
3    FUR-CH-10002024      21870.572     133562.336  1.837761e+06
4    OFF-BI-10001359      19823.484     153385.820  1.837761e+06
..               ...            ...            ...           ...
408  TEC-MA-10002178       1391.400    1831706.505  1.837761e+06
409  TEC-PH-10004434       1386.690    1833093.195  1.837761e+06
410  TEC-PH-10001750       1385.790    1834478.985  1.837761e+06
411  OFF-AP-10002311       1383.081    1835862.066  1.837761e+06
412  TEC-MA-10004212       1379.920    1837241.986  1.837761e+06

[413 rows x 4 columns]
